In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2023
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
import mlflow.spark

In [3]:
import os
import warnings
import sys
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [4]:
import logging
import json
import shutil
import datetime
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
import cml.data_v1 as cmldata

In [5]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
SparkContext.setSystemProperty('spark.executor.cores', '2')
SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/06 00:25:22 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/12/06 00:25:23 WARN JettyUtils: GET /jobs/ failed: java.util.NoSuchElementException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
java.util.NoSuchElementException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
	at org.apache.spark.status.AppStatusStore.applicationInfo(AppStatusStore.scala:51)
	at org.apache.spark.ui.jobs.AllJobsPage.render(AllJobsPa

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|     bri_ranger_demo|
|            cdc_data|
|cde_demo_pauldefusco|
|   cde_demo_pdefusco|
|cde_demo_pdefusco...|
|        cde_workshop|
|cde_workshop_paul...|
+--------------------+
only showing top 20 rows



In [6]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pyspark.sql.types import LongType, IntegerType, StringType, FloatType
from pyspark.sql import functions as F
import dbldatagen as dg
import dbldatagen.distributions as dist
from dbldatagen import FakerTextFactory, DataGenerator, fakerText

class LabeledTextGen:

    '''Class to Generate Text Data'''

    def __init__(self, spark):
        self.spark = spark

    def dataGen(self, shuffle_partitions_requested = 8, partitions_requested = 8, data_rows = 10000):

        # setup use of Faker
        FakerTextUS = FakerTextFactory(locale=['en_US'])

        # partition parameters etc.
        self.spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

        fakerDataspec = (DataGenerator(self.spark, rows=data_rows, partitions=partitions_requested)
                    .withColumnSpec("id", minValue=1, maxValue=data_rows, step=1)
                    .withColumn("text", text=FakerTextUS("address"))
                    .withColumn("label", "string", values=["0", "1"],random=True)
                    )
        df = fakerDataspec.build()
                
        df = df.withColumn("idStr", F.col("id").cast(StringType()))\
            .drop("id")\
            .withColumnRenamed("idStr", "id")
     
        df = df.withColumn("labelStr", F.col("label").cast(FloatType()))\
            .drop("label")\
            .withColumnRenamed("labelStr", "label")
        
        return df

In [7]:
dg = LabeledTextGen(spark)

training_df = dg.dataGen()

In [8]:
training_df.show()

23/12/06 00:25:58 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+--------------------+---+-----+
|                text| id|label|
+--------------------+---+-----+
|USS Garrett\nFPO ...|  0|  1.0|
|74099 Robinson Cu...|  1|  1.0|
|844 Stephens Spur...|  2|  1.0|
|Unit 7442 Box 285...|  3|  1.0|
|526 Matthew Stati...|  4|  1.0|
|907 Gill Park\nSt...|  5|  0.0|
|60684 Ford Viaduc...|  6|  0.0|
|509 Cherry Harbor...|  7|  1.0|
|002 James Fields\...|  8|  1.0|
|2889 Raymond High...|  9|  0.0|
|6508 Brown Mills\...| 10|  1.0|
|052 Emily Branch\...| 11|  1.0|
|7494 Williams Uni...| 12|  1.0|
|2291 Solis Expres...| 13|  0.0|
|5258 Houston Fork...| 14|  0.0|
|314 Sandra Motorw...| 15|  1.0|
|6823 Rodriguez Tr...| 16|  1.0|
|20521 Thompson Gr...| 17|  1.0|
|0285 Ashlee Hill ...| 18|  1.0|
|17266 Williams Sh...| 19|  0.0|
+--------------------+---+-----+
only showing top 20 rows



In [9]:
def exp1(df):

    mlflow.set_experiment("sparkml-experiment")

    ##EXPERIMENT 1

    df.writeTo("spark_catalog.default.training").using("iceberg").createOrReplace()
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    ### SHOW TABLE HISTORY AND SNAPSHOTS
    spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]
    
    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": training_df.count()
    }
    
    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=8
        regParam=0.01

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)
        mlflow.spark.log_model(model, artifact_path="artifacts")

    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)
    
    return runs_df

In [10]:
def exp2(df):
    
    mlflow.set_experiment("sparkml-experiment")
    
    ##EXPERIMENT 2

    ### ICEBERG INSERT DATA - APPEND FROM DATAFRAME

    # PRE-INSERT
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    temp_df = spark.sql("SELECT * FROM spark_catalog.default.training")
    temp_df.writeTo("spark_catalog.default.training").append()
    df = spark.sql("SELECT * FROM spark_catalog.default.training")

    # PROST-INSERT
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]
    
    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": df.count()
    }
    
    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=10
        regParam=0.002

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)
        mlflow.spark.log_model(model, artifact_path="artifacts")

    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)
    
    return runs_df

In [11]:
def exp3(df, snapshot_id):
    ##EXPERIMENT 3

    df = spark.read.option("snapshot-id", snapshot_id).table("spark_catalog.default.training")

    committed_at = spark.sql("SELECT committed_at FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).collect()[0][0].strftime('%m/%d/%Y')
    parent_id = str(spark.sql("SELECT parent_id FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).tail(1)[0][0])

    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": training_df.count()
    }

    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=7
        regParam=0.005

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)
        mlflow.spark.log_model(model, artifact_path="artifacts")#registered_model_name="spark-iceberg-"+username
        
    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)

    #spark.stop()
    
    return runs_df

In [12]:
exp1(training_df)

2023/12/06 00:26:23 INFO mlflow.tracking.fluent: Experiment with name 'sparkml-experiment' does not exist. Creating a new experiment.
23/12/06 00:26:24 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...


+--------------------+---+-----+
|                text| id|label|
+--------------------+---+-----+
|24041 Amy Manor S...|  0|  1.0|
|257 Emily Road\nW...|  1|  1.0|
|04629 Lin Heights...|  2|  1.0|
|440 Charles Roads...|  3|  1.0|
|PSC 4956, Box 834...|  4|  1.0|
|11339 Daniel Mano...|  5|  0.0|
|101 Pierce Mill\n...|  6|  0.0|
|60555 Patrick For...|  7|  1.0|
|9710 Solis Center...|  8|  1.0|
|77892 Lauren Gard...|  9|  0.0|
|59650 Carlson Bro...| 10|  1.0|
|6837 Rodriguez Dr...| 11|  1.0|
|9260 Tony Oval\nT...| 12|  1.0|
|46254 Porter Orch...| 13|  0.0|
|0954 Keith Burg\n...| 14|  0.0|
|USNS Barry\nFPO A...| 15|  1.0|
|19616 Gilbert Run...| 16|  1.0|
|USNS White\nFPO A...| 17|  1.0|
|2430 Brown Knoll\...| 18|  1.0|
|08848 Soto Garden...| 19|  0.0|
+--------------------+---+-----+
only showing top 20 rows

+-----------------------+-------------------+---------+-------------------+
|made_current_at        |snapshot_id        |parent_id|is_current_ancestor|
+-----------------------+-----

23/12/06 00:26:53 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/06 00:26:53 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/12/06 00:26:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/12/06 00:26:53 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2023/12/06 00:27:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpdac_5t4y/model, flavor: spark), fall back to return ['pyspark==3.2.3', 'pandas<2']. Set logging level to DEBUG to see the full traceback.


,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.maxIter,params.regParam,tags.mlflow.user,tags.iceberg_snapshot_committed_at,tags.mlflow.source.name,tags.row_count,tags.engineID,tags.iceberg_snapshot_id,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.iceberg_parent_id
0,prys-g6fa-skk2-5von,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/pr...,2023-12-06 00:26:36.527987968+00:00,2023-12-06 00:27:09.401999872+00:00,8,0.01,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,10000,s193esgf9l4k8xpp,1374289690540954212,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""prys-g6fa-skk2-5von"", ""artifact_p...",None


In [13]:
exp2(training_df)

+--------------------+---+-----+
|                text| id|label|
+--------------------+---+-----+
|24041 Amy Manor S...|  0|  1.0|
|257 Emily Road\nW...|  1|  1.0|
|04629 Lin Heights...|  2|  1.0|
|440 Charles Roads...|  3|  1.0|
|PSC 4956, Box 834...|  4|  1.0|
|11339 Daniel Mano...|  5|  0.0|
|101 Pierce Mill\n...|  6|  0.0|
|60555 Patrick For...|  7|  1.0|
|9710 Solis Center...|  8|  1.0|
|77892 Lauren Gard...|  9|  0.0|
|59650 Carlson Bro...| 10|  1.0|
|6837 Rodriguez Dr...| 11|  1.0|
|9260 Tony Oval\nT...| 12|  1.0|
|46254 Porter Orch...| 13|  0.0|
|0954 Keith Burg\n...| 14|  0.0|
|USNS Barry\nFPO A...| 15|  1.0|
|19616 Gilbert Run...| 16|  1.0|
|USNS White\nFPO A...| 17|  1.0|
|2430 Brown Knoll\...| 18|  1.0|
|08848 Soto Garden...| 19|  0.0|
+--------------------+---+-----+
only showing top 20 rows



+--------------------+---+-----+
|                text| id|label|
+--------------------+---+-----+
|24041 Amy Manor S...|  0|  1.0|
|257 Emily Road\nW...|  1|  1.0|
|04629 Lin Heights...|  2|  1.0|
|440 Charles Roads...|  3|  1.0|
|PSC 4956, Box 834...|  4|  1.0|
|11339 Daniel Mano...|  5|  0.0|
|101 Pierce Mill\n...|  6|  0.0|
|60555 Patrick For...|  7|  1.0|
|9710 Solis Center...|  8|  1.0|
|77892 Lauren Gard...|  9|  0.0|
|59650 Carlson Bro...| 10|  1.0|
|6837 Rodriguez Dr...| 11|  1.0|
|9260 Tony Oval\nT...| 12|  1.0|
|46254 Porter Orch...| 13|  0.0|
|0954 Keith Burg\n...| 14|  0.0|
|USNS Barry\nFPO A...| 15|  1.0|
|19616 Gilbert Run...| 16|  1.0|
|USNS White\nFPO A...| 17|  1.0|
|2430 Brown Knoll\...| 18|  1.0|
|08848 Soto Garden...| 19|  0.0|
+--------------------+---+-----+
only showing top 20 rows



+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2023-12-06 00:26:33.056|1374289690540954212|null               |true               |
|2023-12-06 00:27:24.389|6833554684804309829|1374289690540954212|true               |
+-----------------------+-------------------+-------------------+-------------------+

+-----------------------+-------------------+-------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2023/12/06 00:27:53 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9v8uap_7/model, flavor: spark), fall back to return ['pyspark==3.2.3', 'pandas<2']. Set logging level to DEBUG to see the full traceback.


,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.maxIter,params.regParam,tags.mlflow.user,tags.iceberg_snapshot_committed_at,tags.mlflow.source.name,tags.row_count,tags.engineID,tags.iceberg_snapshot_id,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.iceberg_parent_id
0,prys-g6fa-skk2-5von,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/pr...,2023-12-06 00:26:36.527987968+00:00,2023-12-06 00:27:09.401999872+00:00,8,0.01,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,10000,s193esgf9l4k8xpp,1374289690540954212,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""prys-g6fa-skk2-5von"", ""artifact_p...",None
1,n671-bg5s-3x8w-rap7,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/n6...,2023-12-06 00:27:31.260659968+00:00,2023-12-06 00:27:53.700999936+00:00,10,0.002,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,20000,s193esgf9l4k8xpp,6833554684804309829,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""n671-bg5s-3x8w-rap7"", ""artifact_p...",1374289690540954212


In [14]:
#Retrieve snapshot_id from Experiments page or above dataframe. Use the Snapshot ID from the first experiment.
snapshot_id = "1374289690540954212"
exp3(training_df, snapshot_id)

2023/12/06 00:28:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp9c6h4pm4/model, flavor: spark), fall back to return ['pyspark==3.2.3', 'pandas<2']. Set logging level to DEBUG to see the full traceback.


,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.maxIter,params.regParam,tags.mlflow.user,tags.iceberg_snapshot_committed_at,tags.mlflow.source.name,tags.row_count,tags.engineID,tags.iceberg_snapshot_id,tags.mlflow.source.git.commit,tags.mlflow.source.type,tags.mlflow.log-model.history,tags.iceberg_parent_id
0,prys-g6fa-skk2-5von,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/pr...,2023-12-06 00:26:36.527987968+00:00,2023-12-06 00:27:09.401999872+00:00,8,0.01,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,10000,s193esgf9l4k8xpp,1374289690540954212,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""prys-g6fa-skk2-5von"", ""artifact_p...",None
1,n671-bg5s-3x8w-rap7,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/n6...,2023-12-06 00:27:31.260659968+00:00,2023-12-06 00:27:53.700999936+00:00,10,0.002,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,20000,s193esgf9l4k8xpp,6833554684804309829,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""n671-bg5s-3x8w-rap7"", ""artifact_p...",1374289690540954212
2,em00-97p4-zqv3-bmv1,ebzu-jwz9-3phv-j655,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/ebzu-jwz9-3phv-j655/em...,2023-12-06 00:28:08.274702080+00:00,2023-12-06 00:28:22.784000+00:00,7,0.005,pauldefusco,12/06/2023,/usr/local/lib/python3.9/site-packages/ipykern...,10000,s193esgf9l4k8xpp,1374289690540954212,3ec8d8db6316f35f2bd5bda43dd8d2404b8fbc79,LOCAL,"[{""run_id"": ""em00-97p4-zqv3-bmv1"", ""artifact_p...",None


23/12/06 01:15:57 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
io.fabric8.kubernetes.client.WatcherException: too old resource version: 54174201 (54178994)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onStatus(AbstractWatchManager.java:265)
	at io.fabric8.kubernetes.client.dsl.internal.AbstractWatchManager.onMessage(AbstractWatchManager.java:249)
	at io.fabric8.kubernetes.client.dsl.internal.WatcherWebSocketListener.onMessage(WatcherWebSocketListener.java:93)
	at okhttp3.internal.ws.RealWebSocket.onReadMessage(RealWebSocket.java:323)
	at okhttp3.internal.ws.WebSocketReader.readMessageFrame(WebSocketReader.java:219)
	at okhttp3.internal.ws.WebSocketReader.processNextFrame(WebSocketReader.java:105)
	at okhttp3.internal.ws.RealWebSocket.loopReader(RealWebSocket.java:274)
	at okhttp3.internal.ws.RealWebSocket$2.onResponse(RealWebSocket.java:214)
	at okhttp3.RealCall$AsyncCall.execute(Re